# Responsum notebook

# Справка
Для работы с API Responsum нужно владеть некоторым количеством терминов и представлений. Ниже описаны основные из них

### facility - установка
Установка Responsum определяет период доступных данных, частоту обновления и набор доступных проектов

Подробное описание https://api.mediascope.net/docs/tasks.html


Возможные варианты:

- desktop
- mobile
- desktop-pre

### usetype - тип пользования интернетом

Возможные варианты:

- 1 - desktop
- 2 - mobile-web
- 3 - mobile-app online
- 4 - mobile-app offline


## Доступные варианты отчетности


### Audience - расчет аудиторных статистик

Доступы следующие статистики

- ADF - Average Daily Frequency
- ADO - Average Daily OT
- ADR - Average Daily Reach
- ADRPer - Average Daily Reach %
- Affinity
- AffinityIn - Affinity Internet
- AMF - Average Monthly Frequency
- AMO - Average Monthly OTS
- AMR - Average Monthly Reach
- AMRPer - Average Monthly Reach %
- AvAge - Average Age
- AWDR - Average Weekly Days Reached
- AWF - Average Weekly Frequency
- AWO - Average Weekly OTS
- AWR - Average Weekly Reach
- AWRPer - Average Weekly Reach %
- DR - Days Reached
- DRFD - Days Reached Frequency Distribution
- ExclUseOTSN
- ExclUseReachN
- ExclusiveOts - Имеется в виду эксклюзивность по отношению к медиа
- Frequency
- GRP
- OTS - Opportunity To See. 
- Reach - Охват
- ReachN - Охват и OTS
- Reach_X - количество человек, посетивших страницы интернет-проекта заданное число раз.
- OTS_X - количество загрузок страниц интернет-проекта заданное число раз в соответствии со шкалой разбивки в структуре задания.
- ReachPer
- UnwReach - Unweighted Reach - общее число респондентов

### Duplication - расчет пересечения аудитории медиа-ресурсов

- Reach - Охват
- ADR - Average Daily Reach
- ADRPer - Average Daily Reach %
- AWR - Average Weekly Reach
- AWRPer - Average Weekly Reach %
- AMR - Average Monthly Reach
- AMRPer - Average Monthly Reach %
- UnwReach - Unweighted Reach
- OTS - Opportunity To See
- DR - Days Reached
- Uni - Universe
- Smp - Sample

### Duration - расчет статистик по длительности

- ADDperU
- ADDperP
- ATT

# Настройки

### Логин и пароль пользователя
Для работы с API у вас должен логин и пароль.

_Хранить пароль напрямую в ноутбуке небезопасно, мы будем хранить его в отдельном файле настроек, рядом с ноутбуком,_

Откройте файл: __"settings.json"__ и укажите ваш логин и пароль:
```json
{
	"username": "you login",
	"passw": "you password"
}
```

## Константы и переменные
Для похожих отчетов часто используются одни и те же параметры, удобно их вынести в отдельные переменные, и затем использовать в коде. Если вдруг понадобится какой-то параметр изменить, его нужно поменять в одном месте.


In [2]:
facility = 'mobile'

# Инициализация

Выполните следующую ячейку, чтобы загрузить вспомогательные модули, которые позволят работать с API.

Для этого перейдите в нее и нажмите Ctrl+Enter

In [3]:
%reload_ext autoreload
%autoreload 2
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from resp import catalogs as rc
from resp import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# получаем список демографических переменных
demo_ds = rc.get_demo()
demo_dict = rc.get_demo_dict(demo_ds)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()


Loading BokehJS ...

# Cправочники

## Демографические переменные
Для работы с демографическими переменными реализован метод:
```
get_demo()
```
он принимает пераметры:

* id - Идентификатор демографической переменной для того, чтобы получить одну переменную. По умолчанию - не задано (None).
* find_text - Текст для поиска по названию переменной. По умолчанию - не задано (None).
* expand - Развернуть варианты в таблице? По умолчанию - False.
* frmt - Формат вывода результат: "df" - DataFrame, "json" - JSON, По умолчанию - "df".

Возвращает список демографических переменных.

Вызов метода без параметров выведет список всех демографических переменных:

In [4]:
rc.get_demo()

,varId,varName,varTitle,categories,from,to
0,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None
1,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",2006-09-25,2019-01-31
2,157,Mat_pol,Материальное положение семьи,"[{'varId': 157, 'catNum': 1, 'title': 'не хват...",2006-09-25,None
3,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None
4,161,PERSNUM,Количество человек в семье,"[{'varId': 161, 'catNum': 1, 'title': '1', 'or...",2012-01-19,None
5,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None
6,171,INCOME_GROUPS,Уровень дохода семьи,"[{'varId': 171, 'catNum': 1, 'title': 'Ниже ср...",2006-09-25,None
7,172,EDUC2,Образование,"[{'varId': 172, 'catNum': 1, 'title': 'н.средн...",2006-09-25,None
8,173,WORK2,Занятость,"[{'varId': 173, 'catNum': 1, 'title': 'работае...",2006-09-25,None
9,175,OCCUPATION2,Род занятий,"[{'varId': 175, 'catNum': 1, 'title': 'руковод...",2006-09-25,None


Если передать в метод идентификатор переменной:
```
id = 350
```
или текст для поиска по названию переменной:
```
find_text = 'город'
```
то отобразятся переменные попавшие под фильтр.

Если добавить:
```
expand=True
```
то отобраятся и доступные варианты для переменных.

Следующие две ячейки демонстрируют это:

In [5]:
rc.get_demo(id=173, expand=True)

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,173,WORK2,Занятость,"[{'varId': 173, 'catNum': 1, 'title': 'работае...",2006-09-25,None,1,работает
1,173,WORK2,Занятость,"[{'varId': 173, 'catNum': 1, 'title': 'работае...",2006-09-25,None,2,не работает


In [5]:
rc.get_demo(id=170, expand=True)

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,1,12-17
1,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,2,18-24
2,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,3,25-34
3,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,4,35-44
4,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,5,45-54
5,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,6,55-64
6,170,AGE_GROUPS,Возрастные группы,"[{'varId': 170, 'catNum': 1, 'title': '12-17',...",2006-09-25,None,7,65+


In [6]:
rc.get_demo(find_text='SEX', expand=True)

,varId,varName,varTitle,categories,from,to,catNum,catTitle
0,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,1,Мужчины
1,106,SEX,Пол,"[{'varId': 106, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,2,Женщины
2,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,1,Мужчины 12-17
3,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,2,Мужчины 18-24
4,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,3,Мужчины 25-34
5,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,4,Мужчины 35-44
6,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,5,Мужчины 45-54
7,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,6,Женщины 12-17
8,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,7,Женщины 18-24
9,159,SEXAGE,Пол / Возраст,"[{'varId': 159, 'catNum': 1, 'title': 'Мужчины...",2006-09-25,None,8,Женщины 25-34


In [7]:
rc.get_demo(find_text='город', expand=False)

,varId,varName,varTitle,categories,from,to
0,107,CITY,Город проживания до 01.02.2019,"[{'varId': 107, 'catNum': 1, 'title': 'Москва'...",2006-09-25,2019-01-31
1,246,CITY_TYPE,Численность населения города до 02.02.2014,"[{'varId': 246, 'catNum': 1, 'title': '800 тыс...",2006-09-25,2014-02-02
2,350,CITY_TYPE2,Численность населения города,"[{'varId': 350, 'catNum': 1, 'title': '700 тыс...",2014-02-02,None
3,719,CITY_BM,Город проживания,"[{'varId': 719, 'catNum': 1, 'title': 'Большая...",2019-02-01,None


In [ ]:
rc.get_demo(find_text='город', expand=False)

## Медиа объекты

TODO

- как искать холдинги, сайты, секции, сабсекции
- загрузка медиа-дерава, для чего нужна

In [12]:
df_holdings = rc.get_holdings(facility_id=facility,find_text='ivi.ru')
df_holdings

,id,title,sites,adAgency,network
70,211127,Рекламные кампании Ivi.ru,"[{'id': 363149, 'title': 'Сентябрь 2017', 'sec...",True,False
196,15868,Ivi.ru,"[{'id': 15869, 'title': 'Ivi.ru', 'sections': ...",False,False
533,34496,Рекламные кампании Ivi.ru,"[{'id': 206392, 'title': 'Июль 2014', 'section...",True,False


In [28]:
df_ivi = rc.get_holding(facility_id=facility, id=15868, find_text='ivi.ru')
df_ivi

,id,title,site_id,site_title,section_id,section_title,subsection_id,subsection_title
0,15868,Ivi.ru,15869,Ivi.ru // Программы,41616,Ivi.ru // Программы,332497,Ivi.ru // Программы // Развлекательные
1,15868,Ivi.ru,15869,Ivi.ru // Программы,41616,Ivi.ru // Программы,398629,Ivi.ru // Программы // Игры
2,15868,Ivi.ru,15869,Ivi.ru // Программы,41616,Ivi.ru // Программы,402516,Ivi.ru // Программы // Игры
3,15868,Ivi.ru,15869,Ivi.ru // Программы,41616,Ivi.ru // Программы,453573,Ivi.ru // Программы // Российские
4,15868,Ivi.ru,15869,Ivi.ru // Программы,41616,Ivi.ru // Программы,14799,Ivi.ru // Программы // Красота и здоровье
...,...,...,...,...,...,...,...,...
586,15868,Ivi.ru,17592,Ivi-music.ru // Загрузка ролика,19406,Ivi-music.ru // Загрузка ролика,3400,Ivi-music.ru // Загрузка ролика
587,15868,Ivi.ru,17592,mobile_hidden_17592,214726,mobile_hidden_17592,149498,mobile_hidden_17592
588,15868,Ivi.ru,270991,hidden_270991,270992,hidden_270991,230382,hidden_270991
589,15868,Ivi.ru,270991,mobile_hidden_270991,270993,mobile_hidden_270991,230383,mobile_hidden_270991


In [13]:
df_holdings = rc.get_holdings(facility_id=facility,find_text='avito.ru')
df_holdings

,id,title,sites,adAgency,network
680,377428,Avito.ru (user-centric),"[{'id': 377429, 'title': 'Avito.ru (user-centr...",False,False
2694,15827,Avito.ru,"[{'id': 15828, 'title': 'Avito.ru', 'sections'...",False,False


In [29]:
df_avito = rc.get_holding(facility_id=facility, id=15827)
df_avito

,id,title,site_id,site_title,section_id,section_title,subsection_id,subsection_title
0,15827,Avito.ru,15828,Avito.ru // Хобби и отдых,23442,Avito.ru // Хобби и отдых,1927,Avito.ru // Хобби и отдых (до 30.04.11)
1,15827,Avito.ru,15828,Avito.ru // Хобби и отдых,23442,Avito.ru // Хобби и отдых,5389,Avito.ru // Хобби и отдых // Коллекционирование
2,15827,Avito.ru,15828,Avito.ru // Хобби и отдых,23442,Avito.ru // Хобби и отдых,5390,Avito.ru // Хобби и отдых // Велосипеды
3,15827,Avito.ru,15828,Avito.ru // Хобби и отдых,23442,Avito.ru // Хобби и отдых,5391,Avito.ru // Хобби и отдых // Билеты и путешествия
4,15827,Avito.ru,15828,Avito.ru // Хобби и отдых,23442,Avito.ru // Хобби и отдых,5392,Avito.ru // Хобби и отдых
...,...,...,...,...,...,...,...,...
82,15827,Avito.ru,15828,Avito.ru // Животные,23438,Avito.ru // Животные,5347,Avito.ru // Животные // Корм для животных
83,15827,Avito.ru,15828,Avito.ru // Животные,23438,Avito.ru // Животные,5348,Avito.ru // Животные // Другие животные
84,15827,Avito.ru,15828,Avito.ru // Животные,23438,Avito.ru // Животные,5349,Avito.ru // Животные // Аквариум
85,15827,Avito.ru,15828,Avito.ru // Животные,23438,Avito.ru // Животные,5350,Avito.ru // Животные // Птицы


#### Загрузить медиа-дерево

In [11]:
media_tree = rc.load_mediatree(facility_id=facility, holdings=df_mail, reload=True)
media_tree

,id,title,site_id,site_title,section_id,section_title,subsection_id,subsection_title
0,10,Mail.Ru Group,379948,hidden_379948,379949,hidden_379948,653649,hidden_379948
1,10,Mail.Ru Group,239872,hidden_239872,239873,hidden_239872,182614,hidden_239872
2,10,Mail.Ru Group,239872,Maps.me // total,360418,Maps.me // total,495245,Maps.me // total
3,10,Mail.Ru Group,430017,hidden_430017,430018,hidden_430017,1369707,hidden_430017
4,10,Mail.Ru Group,430017,Hi-chef.ru // total,430020,Hi-chef.ru // total,1359039,Hi-chef.ru // total
...,...,...,...,...,...,...,...,...
2944,373956,Mail.Ru Group видеосеть,373957,Mail.Ru Group видеосеть // total,373960,Mail.Ru Group видеосеть // total,611703,Mail.Ru Group видеосеть // total
2945,373956,Mail.Ru Group видеосеть,373957,hidden_373957,373958,hidden_373957,612836,hidden_373957
2946,409160,Sparkmailapp.com,409161,hidden_409161,409162,hidden_409161,769599,hidden_409161
2947,240322,Aqua-mail.com,240323,hidden_240323,240324,hidden_240323,182867,hidden_240323


# Задания

TODO:

- duration
- автоименование заданий
- сохранение в файл
- вывод моих заданий
- проверка моих заданий - статус

## Задание Audience

Формируем задание

In [23]:
task_name = 'test'
facility = 'desktop'
date_from = '2020-04-01'
date_to = '2020-04-30'
media_filter = "usetype_id = 1 and (site = 12808 or site = 16571)"
demo_filter = "AGE_GROUPS = 1 or AGE_GROUPS = 2"
statistics=["UnwReach", "Reach", "OTS"]
structure =  {
  	"date": "day",
    "media": ["site"],
    "usetype": False
  }

Переводим задание в JSON формат, который понимает Responsum 

In [13]:
task_audience_json = rt.build_audience_task(facility=facility, date_from=date_from, date_to=date_to, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
task_audience_json

'{"header": {"name": "test", "facility": "desktop"}, "filters": {"date": {"from": "2020-04-01", "to": "2020-04-30"}, "media": {"children": [{"point": {"type": "usetype_id", "val": 1}, "operator": "EQUAL", "isNot": false}, {"children": [{"point": {"type": "site", "val": 12808}, "operator": "EQUAL", "isNot": false}, {"point": {"type": "site", "val": 16571}, "operator": "EQUAL", "isNot": false}], "logic": "OR", "isNot": false}], "logic": "AND", "isNot": false}, "demo": {"children": [{"point": {"type": 170, "val": 1}, "operator": "EQUAL", "isNot": false}, {"point": {"type": 170, "val": 2}, "operator": "EQUAL", "isNot": false}], "logic": "OR", "isNot": false}}, "statistics": {"names": ["UnwReach", "Reach", "OTS"]}, "structure": {"date": "day", "media": ["site"], "usetype": false}}'

Отправляем задачу на расчет

In [14]:
task_audience = rt.send_audience_task(task_audience_json)
task_audience

{'profileName': 'responsum_mediascope',
 'taskId': '79c54584-47f4-4d92-b810-b27191d898d1',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <79c54584-47f4-4d92-b810-b27191d898d1> поступила в обработку']}

### Тетсовое задание для ResponsumUI

In [110]:
task_name = 'test#8 (менее 100 тыс.чел.)'
date_from = '2020-05-01'
date_to = '2020-05-31'
media_filter = "site = 15828 or site = 15869)"
demo_filter = "CITY_TYPE2 = 4"
statistics=["Reach"]
structure =  {
    "usetype": False,
    "demo": [159]
  }
#    "media": ["site"],

In [111]:
task_audience_json = rt.build_audience_task(facility=facility, date_from=date_from, date_to=date_to, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
task_audience_json

'{"header": {"name": "test", "facility": "mobile"}, "filters": {"date": {"from": "2020-05-01", "to": "2020-05-31"}, "media": {"children": [{"point": {"type": "site", "val": 15828}, "operator": "EQUAL", "isNot": false}, {"point": {"type": "site", "val": 15869}, "operator": "EQUAL", "isNot": false}], "logic": "OR", "isNot": false}, "demo": {"children": [{"point": {"type": 350, "val": 4}, "operator": "EQUAL", "isNot": false}], "logic": "OR", "isNot": false}}, "statistics": {"names": ["Reach"]}, "structure": {"usetype": false, "demo": [159]}}'

In [112]:
task_audience = rt.send_audience_task(task_audience_json)
task_audience

{'profileName': 'responsum_mediascope',
 'taskId': '422c0a49-771c-48fc-bb61-d7e2013ae921',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <422c0a49-771c-48fc-bb61-d7e2013ae921> поступила в обработку']}

In [113]:
# 6efbcab1-5bb9-4d4b-931e-584983d350dc

audience_task_result = rt.get_result('422c0a49-771c-48fc-bb61-d7e2013ae921')
audience_task_result

{'taskId': '422c0a49-771c-48fc-bb61-d7e2013ae921',
 'profileName': 'responsum_mediascope',
 'deliveryTypes': [],
 'messageType': 'INFO',
 'messages': [],
 'cells': [{'coord': {'dtPoint': None,
    'demoPoint': {'type': 159, 'val': 5},
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'reach': 1395.663991007914}},
  {'coord': {'dtPoint': None,
    'demoPoint': {'type': 159, 'val': 2},
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'reach': 891.9442248683657}},
  {'coord': {'dtPoint': None,
    'demoPoint': {'type': 159, 'val': 1},
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'reach': 988.017508046477}},
  {'coord': {'dtPoint': None,
    'demoPoint': {'type': 159, 'val': 10},
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'reach': 1493.0299144338376}},
  {'coord': {'dtPoint': None,
    'demoPoint': {'type': 159, 'val': 6},
    'mediaPoint': None,
    'usetypePoint': None},
   'values': {'reach': 1089.3974668206781}},


In [100]:
audience_task_table_full = rt.result2table(audience_task_result)
#audience_task_table_full['day'] = pd.to_datetime(audience_task_table_full.day)
audience_task_table_full

,159,reach
0,12,860.422014
1,10,1191.427909
2,9,1928.866484
3,6,653.469827
4,14,438.159173
5,1,513.594018
6,3,1888.840726
7,4,1820.907025
8,7,771.123323
9,5,1055.944639


## Задание Duplication
Формируем задание

__Обратите внимание:__ Демографические переменные нужно задавать названием, а не идентификатором

In [14]:
task_name = 'test-duplication-2'
facility = 'desktop'
date_from = '2020-05-04'
date_to = '2020-05-10'
media_filter = "site = 16571"
dup_media_filter = "site = 12808"
demo_filter = None
statistics=["UnwReach", "Reach", "OTS"]
structure =  {
    "media": ["site"],
    "duplication":["site"],
    "usetype":True,
    "duplicationUsetype": True,
    "date":"week"
}

# Переводим задание в JSON формат, который понимает Responsum.
# Обратите внимание метод для duplication поменялся на build_duplication_task

task_duplication_json = rt.build_duplication_task(task_name=task_name, facility=facility, date_from=date_from, date_to=date_to, 
                                     media_filter=media_filter, dup_media_filter=dup_media_filter, demo_filter=demo_filter,
                                     statistics=statistics, structure=structure)
# Отправляем задание на расчет
task_duplication = rt.send_duplication_task(task_duplication_json)
task_duplication

{'profileName': 'responsum_mediascope',
 'taskId': 'd8044f4e-6db8-47d4-bfe8-2462f6568f61',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <d8044f4e-6db8-47d4-bfe8-2462f6568f61> поступила в обработку']}

## Задание Duration

In [11]:
task_name = 'test-duration'
facility = 'mobile'
date_from = '2020-01-01'
date_to = '2020-02-29'
media_filter = "site = 238816"
demo_filter = None
statistics=["ADDperP", "ADDperU"]
structure =  {
    "media": ["site"],
    "date":"month"
}

# Переводим задание в JSON формат, который понимает Responsum.
# Обратите внимание метод для duration поменялся на build_duration_task

task_duration_json = rt.build_duration_task(task_name=task_name, facility=facility, date_from=date_from, date_to=date_to, 
                                     media_filter=media_filter, demo_filter=demo_filter,
                                     statistics=statistics, structure=structure)
# Отправляем задание на расчет
task_duration = rt.send_duration_task(task_duration_json)
task_duration

{'profileName': 'responsum_mediascope',
 'taskId': 'ce518ab5-a680-411b-8526-8938f966a47c',
 'operation': 'Расчет задачи.',
 'messages': ['Задача <ce518ab5-a680-411b-8526-8938f966a47c> поступила в обработку']}

## Получить результат

In [24]:
#audience_task_result = rt.get_result('30895470-88a6-433d-963e-9ab536d9afbb')
audience_task_result = rt.get_result('79c54584-47f4-4d92-b810-b27191d898d1')
audience_task_result

{'taskId': '79c54584-47f4-4d92-b810-b27191d898d1',
 'profileName': 'responsum_mediascope',
 'deliveryTypes': [],
 'messageType': 'INFO',
 'messages': [],
 'cells': [{'coord': {'dtPoint': {'type': 'day', 'val': '2020-04-01'},
    'demoPoint': None,
    'mediaPoint': {'type': 'site', 'val': 12808},
    'usetypePoint': None},
   'values': {'unwreach': 44.0,
    'reach': 467.31238550905294,
    'ots': 5674.201503489861}},
  {'coord': {'dtPoint': {'type': 'day', 'val': '2020-04-01'},
    'demoPoint': None,
    'mediaPoint': {'type': 'site', 'val': 16571},
    'usetypePoint': None},
   'values': {'unwreach': 593.0,
    'reach': 4152.544185361256,
    'ots': 190441.46926177814}},
  {'coord': {'dtPoint': {'type': 'day', 'val': '2020-04-02'},
    'demoPoint': None,
    'mediaPoint': {'type': 'site', 'val': 12808},
    'usetypePoint': None},
   'values': {'unwreach': 47.0,
    'reach': 468.90196134954795,
    'ots': 7035.796746499154}},
  {'coord': {'dtPoint': {'type': 'day', 'val': '2020-04-02'

In [16]:

task_duration_result = rt.get_result('ce518ab5-a680-411b-8526-8938f966a47c')
#task_duration_result
task_duration_table = rt.result2table(task_duration_result)
task_duration_table

,month,site,add_per_p,add_per_u
0,2020-01-01,238816,0,3.2
1,2020-02-01,238816,0,2.3


Преобразуем результат в таблицу (DataFrame)

In [17]:
audience_task_table_full = rt.result2table(audience_task_result)
audience_task_table_full['day'] = pd.to_datetime(audience_task_table_full.day)
audience_task_table_full

,day,site,unwreach,reach,ots
0,2020-04-01,12808,44.0,467.312386,5674.201503
1,2020-04-01,16571,593.0,4152.544185,190441.469262
2,2020-04-02,12808,47.0,468.901961,7035.796746
3,2020-04-02,16571,603.0,4398.054778,191530.775531
4,2020-04-03,12808,50.0,452.575378,4989.133266
5,2020-04-03,16571,573.0,4124.056095,207438.865413
6,2020-04-04,12808,40.0,428.896977,5823.370007
7,2020-04-04,16571,564.0,4078.082595,181036.548153
8,2020-04-05,12808,50.0,520.364210,9553.389809
9,2020-04-05,16571,566.0,3917.375831,171051.782689


Можно сделать сводную таблицу

In [18]:
audience_task_table_pivot = pd.pivot_table(audience_task_table_full, columns=['site'], index=['day'], values=['unwreach', 'reach', 'ots'])
audience_task_table_pivot

ots                      reach              unwreach  \
site               12808          16571       12808        16571    12808   
day                                                                         
2020-04-01   5674.201503  190441.469262  467.312386  4152.544185     44.0   
2020-04-02   7035.796746  191530.775531  468.901961  4398.054778     47.0   
2020-04-03   4989.133266  207438.865413  452.575378  4124.056095     50.0   
2020-04-04   5823.370007  181036.548153  428.896977  4078.082595     40.0   
2020-04-05   9553.389809  171051.782689  520.364210  3917.375831     50.0   
2020-04-06  10877.162155  212040.635504  582.947861  4856.068197     48.0   
2020-04-07  12664.600075  209423.831150  606.651103  4736.898458     48.0   
2020-04-08  12312.586646  200776.297924  534.397719  4993.360110     45.0   
2020-04-09  14693.363028  196939.758737  516.207474  4832.354375     44.0   
2020-04-10  10234.806203  256686.300381  613.647070  4950.869024     49.0   
2020-04-11   9446.672518  180921.765723  439.653542  4328.291583     38.0   
2020-04-12   7959.528001  166446.238709  673.869638  4518.824364     59.0   
2020-04-13   9082.067147  252063.445393  519.022953  5061.642214     40.0   
2020-04-14   9883.308542  231541.757071  571.659195  4939.135722     48.0   
2020-04-15   9482.512444  228194.678384  697.261639  5066.628111     55.0   
2020-04-16  10131.056794  225284.692973  449.563592  5115.099432     39.0   
2020-04-17  10028.949072  203148.916767  589.664549  4660.515612     47.0   
2020-04-18  13303.464887  156642.190059  539.102639  4466.683188     45.0   
2020-04-19  15623.723692  155970.874719  598.861309  4562.982122     55.0   
2020-04-20  19367.493290  210229.681048  566.191922  4887.387284     49.0   
2020-04-21  10117.997226  200027.733290  425.439401  5253.288283     40.0   
2020-04-22   9094.899611  212979.459178  780.314244  5249.968606     50.0   
2020-04-23  10242.949244  222240.152164  438.900921  5102.451608     43.0   
2020-04-24  10091.469325  187284.500210  633.091689  4620.290850     55.0   
2020-04-25   9404.644572  159616.238792  616.130773  4413.913954     48.0   
2020-04-26   8915.996361  156337.934254  592.179388  3992.545799     50.0   
2020-04-27   9310.498236  210870.328503  600.710188  4945.154628     50.0   
2020-04-28   7417.953122  211295.150815  465.337132  4854.982749     48.0   
2020-04-29  11148.717675  201641.111191  617.230351  5030.068841     47.0   
2020-04-30  10104.364321  187324.867841  491.951655  4729.144919     43.0   

                   
site        16571  
day                
2020-04-01  593.0  
2020-04-02  603.0  
2020-04-03  573.0  
2020-04-04  564.0  
2020-04-05  566.0  
2020-04-06  654.0  
2020-04-07  647.0  
2020-04-08  652.0  
2020-04-09  645.0  
2020-04-10  648.0  
2020-04-11  601.0  
2020-04-12  606.0  
2020-04-13  669.0  
2020-04-14  659.0  
2020-04-15  675.0  
2020-04-16  669.0  
2020-04-17  644.0  
2020-04-18  600.0  
2020-04-19  591.0  
2020-04-20  647.0  
2020-04-21  660.0  
2020-04-22  666.0  
2020-04-23  632.0  
2020-04-24  616.0  
2020-04-25  578.0  
2020-04-26  564.0  
2020-04-27  627.0  
2020-04-28  619.0  
2020-04-29  644.0  
2020-04-30  621.0

### Сразу отобразить результат в таблице
нужно в одной ячейке вызвать последовательно несколько команд

In [19]:
task_duplication_result = rt.get_result('46dc6d2c-7bff-4122-bf87-ba13314cacef')
task_duplication_table = rt.result2table(task_duplication_result)
task_duplication_table

,week,usetype_id,site,unwreach,reach,ots
0,2020-05-04,1,16571,1829.0,9102.885216,994675.375921


# Экспорт в Excel
Все полученные DataFrame'ы можно экспортировать в Excel

In [20]:
Path('data').mkdir(exist_ok=True)

In [21]:
with pd.ExcelWriter('data/my_report.xlsx') as writer:
    audience_task_table_full.to_excel(writer, sheet_name='Audience_full')
    audience_task_table_pivot.to_excel(writer, sheet_name='Audience_pivot')

# Графики

In [22]:
# 16571 - VK
# 12808 - OK
r = audience_task_table_full
r['dt'] = r.day.dt.strftime('%Y-%m-%d')
df_ok = r.loc[r['site']==12808]
df_vk = r.loc[r['site']==16571]

hover = HoverTool(tooltips=[("index", "$index"),("(x,y)", "($x, $y)"),("desc", "@dt"),])
p1 = figure(x_axis_type="datetime", title="VK & OK", plot_height=350, plot_width=960, tools=[hover])
#p.xgrid.grid_line_color=None
p1.ygrid.grid_line_alpha=0.5
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Reach'
p1.xaxis.formatter.days = '%d-%m-%Y'

p1.line('day', 'reach', legend_label="VK.com", source=df_vk)
p1.line('day', 'reach', line_color="orange", legend_label="Odnoklassniki", source=df_ok)


p2 = figure(x_axis_type="datetime", title="VK & OK ", plot_height=350, plot_width=960, tools=[hover])
#p.xgrid.grid_line_color=None
p2.ygrid.grid_line_alpha=0.5
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'OTS'
p2.xaxis.formatter.days = '%d-%m-%Y'

p2.line('day', 'ots', legend_label="VK.com", source=df_vk)
p2.line('day', 'ots', line_color="orange", legend_label="Odnoklassniki", source=df_ok)

p = gridplot([[p1],[p2]])

show(p)
